# B站小黑屋是一种异步加载的网页结构，打开网页不会显示所有内容，当向下滚动时，会再加载新内容

##################################################################################<br>
<font color=black size=5 face="黑体">整体思路就是分为两步：</font>  
<font color=black size=4 face="黑体">（1）完全加载页面</font>  <br>
<font color=black size=4 face="黑体">（2）爬取内容</font>  <br>
##################################################################################<br>

<font color=#0099ff size=5 face="黑体">经过观察，每个黑评的xpath有规律可循 </font>  

/html/body/div[3]/div/div/div/div[2]/div[2]/<font color=#0099ff >div[1]/div[1]</font>/a[2]<br>
/html/body/div[3]/div/div/div/div[2]/div[2]/<font color=#0099ff >div[2]/div[1]</font>/a[2]<br>
/html/body/div[3]/div/div/div/div[2]/div[2]/<font color=#0099ff >div[3]/div[1]</font>/a[2]<br>
由上到下依次是：第一列第一行数据，第二列第一行数据，第三列第一行数据<br>
每次加载最多4行（大约）<br>

其中，针对<font color=red >恶意投稿</font>和<font color=red >弹幕</font>的处理结果格式不一样，需要特别处理

<font color=grey size=5 face="黑体">V1.0   不处理数据，仅爬取，格式混乱，需要后期处理</font>  

In [1]:
from selenium import webdriver
import pandas as pd
import re
import time

In [4]:
#加载网页
def web_load(driver, count):
    for i in range(0,count):
        #print(i)
        js = "window.scrollTo(0,document.body.scrollHeight)"
        driver.execute_script(js)
        time.sleep(5)

In [5]:
#遍历行列
def get_div_content(driver, count, df):
    row = count * 4 + 1
    col = 3 + 1
    i = 0
    #xpath
    str1 = """/html/body/div[3]/div/div/div/div[2]/div[2]/div["""
    str2 = """]/div["""
    str3 = """]/a[2]"""
    
    for r in range(1,row):
        for c in range(1, col):
            try:
                s = driver.find_element_by_xpath(str1 + str(c)+ str2 + str(r) + str3).text
            except:
                continue
            else:
                #print(r,c)
                df.loc[i] = s
                i += 1
                 
    return df

In [6]:
#打开火狐浏览器
driver = webdriver.Firefox(executable_path = 'geckodriver')
#指定网址
url = "https://www.bilibili.com/blackroom/ban"
#打开网页
driver.get(url)
time.sleep(4)
#指定滚动次数or加载次数
count = 2  
web_load(driver, count)
df = pd.DataFrame(columns=['s'])
new_df = get_div_content(driver, count, df)

In [7]:
new_df.head()

,s
0,我就是性格如此\n在评论中发布引战言论封禁 7 天\n不投币的高考必定发挥失常，大学生必定考...
1,飘零的蒲草\n在评论中发布人身攻击言论封禁 30 天\n举报我的没有亲妈。看不惯吗，我就是要...
2,福煦刚刚\n在评论中发布引战言论封禁 7 天\n真搞不懂看直播就看直播有必要刷礼物吗杀b们\...
3,P90暴力RB\n在评论中发布引战言论封禁 7 天\n我真服了，一个几十秒就能说完的事让这个...
4,灼魂双面吨\n在评论中发布垃圾广告信息永久封禁\n陌生人 早点睡觉吧 真的睡不着的话 DIA...


In [13]:
new_df.to_excel('B站小黑屋评论V1.0.xlsx')